In [5]:

! pip install azure-search-documents

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchableField,
    SearchFieldDataType
)
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import os

# Configuration
endpoint = os.getenv("SEARCH_END_POINT")
admin_key = os.getenv("SEARCH_ADMIN_KEY")  # Replace with your environment variable setup
index_name = os.getenv("SEARCH_INDEX_NAME")

# Initialize the SearchIndexClient
index_client = SearchIndexClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(admin_key)
)

# Retrieve the existing index
index = index_client.get_index(index_name)

# Define the new searchable field with a search analyzer
new_field = SearchableField(
    name="title3",
    type=SearchFieldDataType.String,
    analyzer="en.lucene",  # Specify the desired search analyzer
    searchable=True,
    filterable=True,
    sortable=True,
    facetable=True
)

# Check if the new field already exists to avoid duplication
if any(field.name == new_field.name for field in index.fields):
    print(f"Field '{new_field.name}' already exists in the index.")
else:
    # Add the new field to the index
    index.fields.append(new_field)

    # Update the index
    index_client.create_or_update_index(index)
    print(f"Field '{new_field.name}' added to the index '{index_name}' with search analyzer.")

    # Initialize the SearchClient for document operations
    search_client = SearchClient(
        endpoint=endpoint,
        index_name=index_name,
        credential=AzureKeyCredential(admin_key)
    )

    # Retrieve all documents
    results = search_client.search(search_text="*", include_total_count=True)
    
    # Prepare updated documents
    updated_documents = []
    for doc in results:
        doc['title3'] = doc.get('title')
        updated_documents.append(doc)

    # Upload updated documents
    search_client.merge_or_upload_documents(documents=updated_documents)
    print("Populated 'title3' with existing 'title' data.")

Field 'title3' added to the index 'aoaipilotrmindex' with search analyzer.
Populated 'title3' with existing 'title' data.
